# Using Amazon Bedrock's OpenAI Compatible API with OSS Agentic Frameworks

This notebook demonstrates how to leverage the OpenAI compatible API on Amazon Bedrock with popular open source frameworks including Strands Agents, LangChain/LangGraph, and CrewAI. We'll explore how these frameworks can seamlessly work with Bedrock's OpenAI-compatible endpoint.
## What You'll Learn

- Setting up OpenAI compatible API with Amazon Bedrock
- Integrating Strands Agents with Bedrock
- Using LangChain/LangGraph with Bedrock's OpenAI endpoint
- Implementing CrewAI multi-agent workflows on Bedrock
- Best practices for framework integration

## Prerequisites

Ensure you have:
- Access to Amazon Bedrock with OpenAI models enabled
- AWS credentials configured
- Python environment with required packages

## Environment Setup

First, let's install the required packages and configure the environment.

In [ ]:
%pip install -U boto3 openai langchain langgraph crewai strands-agents

In [ ]:
import os
import json
import boto3
from datetime import datetime
from openai import OpenAI
from IPython.display import display, Markdown

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

# LangGraph imports
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated

# CrewAI imports
from crewai import Agent, Task, Crew, Process

# Strands imports
from strands import Agent as StrandsAgent
from strands.models.bedrock import BedrockModel

print("✅ All packages imported successfully!")

## Amazon Bedrock Configuration

Configure the environment to use Amazon Bedrock's OpenAI compatible endpoint.

In [ ]:
# Model Configuration
MODEL_ID = "gpt-oss-120b"  # or "gpt-oss-20b" for faster inference

# Set region to use for Bedrock
target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION", "us-west-2"))

# Set environment variables for OpenAI SDK to point to Bedrock
os.environ["OPENAI_BASE_URL"] = f'https://bedrock-runtime.{target_region}.amazonaws.com/openai/v1'
os.environ["OPENAI_API_KEY"] = "<insert-your-bedrock-api-key>"

# Initialize OpenAI client (pointing to Bedrock)
openai_client = OpenAI()


print(f"✅ Using model: {MODEL_ID}")
print("✅ OpenAI client configured for Bedrock")
print("✅ Bedrock client initialized")

## 1. Strands Agents with Amazon Bedrock

Strands is AWS's open-source agent framework that provides single-file agents, built-in multi-agent hand-offs, and first-class Bedrock support.

### Model setup

We start by initializing the model wrapper class `BedrockModel`.

In [ ]:
# Initialize Bedrock model for Strands
bedrock_model = BedrockModel(
    model_id=MODEL_ID,
    region="us-west-2"
)

print(f"✅ Strands Bedrock model initialized: {MODEL_ID}")

### Summarizer-Triager Agent

We build a simple agentic system consisting of two agents that summarize and then triage customer support tickets. This demonstrates OpenAI OSS model's elevated capabilities for building agentic applications.

In [ ]:
summarizer = StrandsAgent(
    model=bedrock_model,
    system_prompt=(
       "You are SupportSummarizer. "
        "For each ticket in the list you receive:\n"
        "  * Produce JSON with keys id, summary (≤25 words), severity (low|medium|high)\n"
        "Return **only** a JSON array."
    )
)

triager = Agent(
    model=bedrock_model,
    system_prompt=(
        "You are TriageBot. You get a JSON array of tickets that already "
        "contain severity. For every item output:\n"
        "  * id (same as input)\n"
        "  * route = 'escalate' if severity is high, else 'backlog'\n"
        "Respond with **only** a JSON array."
    ),
)

# Test the agent
tickets = [
    {"id": "1", "text": "User cannot log in after resetting password – URGENT, blocks payroll."},
    {"id": "2", "text": "Minor typo in footer on marketing site."},
]

# summarise
json_summaries = summarizer([t["text"] for t in tickets]).content   

# triage
routes = triager(json_summaries).content

print("Summaries:", json_summaries)
print("Routes   :", routes)

### Analyzer-Reporter Agent

OpenAI OSS models are also optimized for reasoning in agentic applications. To demonstrate this, we set up a Strands Multi-Agent system for analyzing data and creating structured reports.

In [ ]:
analyzer_agent = StrandsAgent(
    model=bedrock_model,
    system_prompt=(
        "You are a DataAnalyzer. Analyze the given data and extract key insights. "
        "Return your analysis in JSON format with keys: 'insights', 'trends', 'recommendations'."
    )
)

reporter_agent = StrandsAgent(
    model=bedrock_model,
    system_prompt=(
        "You are a ReportWriter. Take the analysis results and create a clear, "
        "professional report. Focus on actionable insights and next steps."
    )
)

# Sample data for analysis
sample_data = """
Monthly Performance Data:
- January: 85% customer satisfaction, 1200 users
- February: 87% customer satisfaction, 1350 users  
- March: 89% customer satisfaction, 1500 users
- April: 91% customer satisfaction, 1650 users
- May: 93% customer satisfaction, 1800 users
"""

# Multi-agent workflow
print("📊 Running Multi-Agent Analysis...")
print("=" * 50)

# Step 1: Analyze data
analysis = analyzer_agent(sample_data)
print("📈 Analysis Results:")
print(analysis.content)
print("\n" + "=" * 50)

# Step 2: Generate report
report = reporter_agent(analysis.content)
print("📋 Final Report:")
print(report.content)

## 2. LangChain and LangGraph with Amazon Bedrock

### Using LangChain's OpenAI wrapper classes natively with Amazon Bedrock

LangChain provides a comprehensive framework for building LLM applications. We'll explore how to use LangChain's popular OpenAI wrapper classes with Bedrock's OpenAI compatible endpoint. 

**Note:** The environment variables `OPENAI_BASE_URL` and `OPENAI_API_KEY` set in the Amazon Bedrock Configuration section above will redirect traffic to Amazon Bedrock's OpenAI compatible endpoint and handle authentication via the Amazon Bedrock API key. Alternatively, these settings can be specified directly in the `ChatOpenAI` wrapper class through the parameters `base_url` and `api_key`.

In [ ]:
# Initialize LangChain with Bedrock
langchain_llm = ChatOpenAI(
    model=MODEL_ID,
    temperature=0.1,
    max_tokens=1000
)

print(f"✅ LangChain LLM initialized with model: {MODEL_ID}")

In [ ]:
# Simple LangChain Chain
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. Provide clear, concise explanations."),
    ("human", "Explain the concept of {topic} in programming.")
])

chain = LLMChain(llm=langchain_llm, prompt=prompt_template)

# Test the chain
response = chain.run(topic="dependency injection")

print("💻 LangChain Chain Response:")
print("=" * 50)
print(response)

### Building a LangGraph Workflow with LangChain's OpenAI Wrapper Classes and Amazon Bedrock

LangGraph enables building complex, stateful workflows with LLMs. Let's create a multi-step workflow utilizing LangChain's OpenAI wrapper classes as demonstrated above. Our sample workflow will research a topic, analyze the results, and create a concise summary.

Behind the scenes, this implementation utilizes Bedrock's OpenAI compatible endpoint.


**Note:** The environment variables `OPENAI_BASE_URL` and `OPENAI_API_KEY` set in the Amazon Bedrock Configuration section above will redirect traffic to Amazon Bedrock's OpenAI compatible endpoint and handle the authentication via Amazon Bedrock API key. Alternatively, these settings can be specified directly in the `ChatOpenAI` wrapper class through the parameters `base_url` and `api_key`. 

In [ ]:
# Define state structure
class WorkflowState(TypedDict):
    query: str
    research: str
    analysis: str
    summary: str

# Initialize LangGraph LLM
langgraph_llm = ChatOpenAI(
    model=MODEL_ID,
    temperature=0.1
)

print("✅ LangGraph setup complete")

In [ ]:
# Define workflow functions
def research_topic(state: WorkflowState) -> WorkflowState:
    """Research the given topic"""
    prompt = f"Research the following topic: {state['query']}. Provide comprehensive information."
    response = langgraph_llm.invoke([HumanMessage(content=prompt)])
    state['research'] = response.content
    return state

def analyze_research(state: WorkflowState) -> WorkflowState:
    """Analyze the research findings"""
    prompt = f"Analyze this research: {state['research']}. Identify key insights and patterns."
    response = langgraph_llm.invoke([HumanMessage(content=prompt)])
    state['analysis'] = response.content
    return state

def create_summary(state: WorkflowState) -> WorkflowState:
    """Create a final summary"""
    prompt = f"Create a concise summary based on: Research: {state['research']}, Analysis: {state['analysis']}"
    response = langgraph_llm.invoke([HumanMessage(content=prompt)])
    state['summary'] = response.content
    return state

print("✅ Workflow functions defined")

In [ ]:
# Build the workflow graph
workflow = StateGraph(WorkflowState)

# Add nodes
workflow.add_node("research", research_topic)
workflow.add_node("analyze", analyze_research)
workflow.add_node("summarize", create_summary)

# Add edges
workflow.add_edge("research", "analyze")
workflow.add_edge("analyze", "summarize")
workflow.add_edge("summarize", END)

# Compile the workflow
app = workflow.compile()

print("✅ LangGraph workflow compiled")

In [ ]:
# Execute the workflow
initial_state = {
    "query": "The impact of artificial intelligence on healthcare"
}

print("🔄 Executing LangGraph Workflow...")
print("=" * 50)

result = app.invoke(initial_state)

print("📋 Final Summary:")
print("=" * 50)
print(result['summary'])

print("\n📊 Analysis:")
print("=" * 50)
print(result['analysis'])

## 3. CrewAI with Amazon Bedrock

CrewAI enables building multi-agent systems where agents can collaborate on complex tasks. Let's create a multi-agent crew utilizing CrewAI's `LLM` wrapper class. Our sample crew will research a topic, write content based on the results, and perform a final quality review.

Behind the scenes, this implementation utilizes Bedrock's OpenAI compatible endpoint.

**Note:** The environment variables `OPENAI_BASE_URL` and `OPENAI_API_KEY` set in the Amazon Bedrock Configuration section above will redirect traffic to Amazon Bedrock's OpenAI compatible endpoint and handle the authentication via Amazon Bedrock API key. 

In [ ]:
# Configure CrewAI to use Bedrock
from crewai import Agent, Task, Crew, LLM

crewai_llm = LLM(
    model=f'openai/MODEL_ID', 
    temperature=0.1
)

print(f"✅ CrewAI LLM initialized with model: {MODEL_ID}")

In [ ]:
# Define specialized agents
researcher = Agent(
    role='Research Analyst',
    goal='Conduct thorough research on given topics',
    backstory='Expert at gathering and analyzing information from various sources',
    verbose=True,
    allow_delegation=False,
    llm=crewai_llm
)

writer = Agent(
    role='Content Writer',
    goal='Create engaging and informative content based on research',
    backstory='Skilled writer with expertise in creating compelling narratives',
    verbose=True,
    allow_delegation=False,
    llm=crewai_llm
)

reviewer = Agent(
    role='Quality Reviewer',
    goal='Ensure content quality, accuracy, and completeness',
    backstory='Experienced editor with keen eye for detail and quality assurance',
    verbose=True,
    allow_delegation=False,
    llm=crewai_llm
)

print("✅ CrewAI agents defined")

In [ ]:
# Define tasks for the crew
research_task = Task(
    description=(
        "Research the topic: 'Sustainable AI Development Practices'. "
        "Focus on environmental impact, ethical considerations, and best practices. "
        "Provide comprehensive findings with key insights."
    ),
    agent=researcher,
    expected_output="Detailed research findings with key insights and data points"
)

writing_task = Task(
    description=(
        "Based on the research findings, create a comprehensive article about "
        "Sustainable AI Development Practices. The article should be engaging, "
        "informative, and suitable for a technical audience."
    ),
    agent=writer,
    expected_output="A well-structured article with introduction, main sections, and conclusion"
)

review_task = Task(
    description=(
        "Review the article for accuracy, completeness, and quality. "
        "Ensure it meets professional standards and provides valuable insights. "
        "Suggest improvements if needed."
    ),
    agent=reviewer,
    expected_output="Quality review with feedback and suggestions for improvement"
)

print("✅ CrewAI tasks defined")

In [ ]:
# Create and execute the crew
crew = Crew(
    agents=[researcher, writer, reviewer],
    tasks=[research_task, writing_task, review_task],
    verbose=True,
    process=Process.sequential
)

print("🚀 Executing CrewAI Workflow...")
print("=" * 50)

result = crew.kickoff()

print("📄 Final Article:")
print("=" * 50)
print(result)

# Conclusion

## What We've Accomplished

You've successfully integrated four major open source agentic frameworks with Amazon Bedrock's OpenAI compatible API:

- ✅ **Strands Agents**: AWS-native agent framework with multi-agent workflows
- ✅ **LangChain**: Comprehensive LLM application development with chains and tools  
- ✅ **LangGraph**: Stateful workflow orchestration for complex processes
- ✅ **CrewAI**: Multi-agent collaboration for specialized task decomposition

## Key Takeaways

- 🎯 **Unified API**: All frameworks work seamlessly with Bedrock's OpenAI endpoint
- 🚀 **Performance**: Each framework optimized for different use cases
- 🔧 **Developer Experience**: Familiar LLM wrappers integrate with Amazon Bedrock, enhancing developer experience and reducing migration efforts
- 💰 **Cost Efficiency**: Competitive pricing with enterprise reliability


The OpenAI compatible API on Amazon Bedrock provides unprecedented flexibility for building sophisticated AI applications while maintaining familiar patterns and ecosystems.